In [2]:
# -*- coding: UTF-8 -*-

#import the necessary libraries
from bs4 import BeautifulSoup
import re
import pandas as pd
import urllib.request
from urllib.request import urlopen

In [4]:
def webpage(page):
    url = 'https://3rdguide.com/web/stig/detail?id=' + str(page)
    html = urlopen(url).read()
    html = html.decode('utf-8')
    sup = BeautifulSoup(html,'html.parser')
    return compiling(sup)

In [6]:
def get_setname(sup):
    tag = 'p'
    clas = "mark-item-16 center"
    html = sup.find(tag,class_=clas)
    setname = html.get_text()
    return setname

In [8]:
def get_stig(sup):
    tag = 'div'
    clas = 'stig-table'
    stiglist = []
    df = pd.DataFrame()
    sr = pd.DataFrame()
    hp = None
    atk = None
    dfs = None
    crt = None
    e = 1
    for element in sup.find_all(tag,class_=clas):
        text = element.get_text().split("\n\n\n")
        stigname = text[1]
        stigstat = list(text[2].split("\n"))
        hp = stigstat[0]
        atk = stigstat[1]
        dfs = stigstat[2]
        crt = stigstat[3]
        dic = {
            e:[stigname,hp,atk,dfs,crt]
        }
        df = df.append(pd.DataFrame.from_dict(dic,orient='index',columns=['部位','生命','攻擊','防禦','會心']))
        e = e + 1
    df = df.reset_index(drop=True)
    return df

In [10]:
def set_setname(sup):
    df = pd.Series(dtype=str)
    instances = len(get_stig(sup))
    result = []
    for i in range(instances):
        result.append(get_setname(sup))
    return result

In [12]:
def get_desc(sup):
    tag = 'p'
    clas = 'mark-item-19'
    result = []
    sr = pd.Series(dtype=str)
    for element in sup.find_all(tag,class_=clas):
        text = element.get_text()
        text = text.strip()
        sr = sr.append(pd.Series(text,dtype=str))
    sr = sr[1:]
    return sr

In [14]:
def frame():
    dic_desc = {
        '套裝':set_setname(),
        '能力':get_desc()
    }
    df = pd.DataFrame.from_dict(dic_desc,dtype=str).reset_index(drop=True)
    df = df.join(get_stig(),how='left')
    df = df.filter(items=['套裝','部位','生命','攻擊','防禦','會心','能力'])
    return df

In [25]:
def setability(sup):
    settwo = sup.find('p',class_='mark-item-20').get_text()
    setthree = sup.find('p',class_='mark-item-20 mb60').get_text()
    if len(setthree) == None and len(settwo) == None:
        result = None
        return result
    elif len(setthree) == None and len(settwo) != None:
        result = pd.DataFrame(
            data=[get_setname(sup)[0],'兩件套','','','','',settwo],
            columns=['套裝','部位','生命','攻擊','防禦','會心','能力']
            )
        return result
    elif len(setthree) != None and len(settwo) != None:
        result = pd.DataFrame(
            data=[[get_setname(sup)[0],'兩件套','','','','',settwo.strip()],[get_setname(sup)[0],'三件套','','','','',setthree.strip()]],
            columns=['套裝','部位','生命','攻擊','防禦','會心','能力']
        )
        return result

In [21]:
def compiling(sup):
    if type(setability(sup)) == str:
        df = frame(sup)
        return df
    elif type(setability(sup)) != str:
        df = pd.concat([frame(sup),setability(sup)],axis=0)
        return df

In [ ]:
def get_info(pages):
    df = pd.DataFrame()
    r = range(pages)
    for page in r:
        result = webpage(page)
        df = pd.concat([df,result],axis=index)
    return df

In [28]:
get_info(10)

AttributeError: 'NoneType' object has no attribute 'get_text'